In [1]:
import spacy
import pandas as pd
import re
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer
from deep_translator import GoogleTranslator
from typing import Any, Dict, List
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#load main models
nlp_en = spacy.load("en_core_web_lg")
nlp_de = spacy.load("de_core_news_lg")

model_cache = {}

In [2]:
def combine_text_columns(
    df: pd.DataFrame,
    columns_to_combine: List[str],
    output_column: str = "CombinedText"
) -> pd.DataFrame:
    """
    Combine multiple text columns into a single new column in the DataFrame.

    This function safely handles missing values and whitespace.

    Args:
        df (pd.DataFrame): The input DataFrame containing the text columns.
        columns_to_combine (List[str]): List of column names to combine.
        output_column (str): Name of the new combined column to create.

    Returns:
        pd.DataFrame: The original DataFrame with an additional combined text column.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")

    if not isinstance(columns_to_combine, list) or not all(isinstance(col, str) for col in columns_to_combine):
        raise TypeError("columns_to_combine must be a list of column name strings.")

    missing_cols = [col for col in columns_to_combine if col not in df.columns]
    if missing_cols:
        raise ValueError(f"The following columns are missing from the DataFrame: {missing_cols}")

    df["Combined_Text_Col"] = df[columns_to_combine]\
        .fillna("")\
        .astype(str)\
        .agg(" ".join, axis=1)\
        .str.strip()

    return df

def load_marian_model(src_lang: str):
    """
    Load MarianMT model and tokenizer for a given source language.
    """
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-en"
    if model_name not in model_cache:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        model_cache[model_name] = (tokenizer, model)
    return model_cache[model_name]

def translate_to_english(text: Any) -> str:
    """
    Detect source language and translate text to English using MarianMT.

    Args:
        text (Any): Input text.

    Returns:
        str: English translation (or original if already English or untranslatable).
    """
    if not isinstance(text, str):
        return ""

    try:
        lang = detect(text)
    except Exception:
        return text  # fallback if language detection fails

    if lang == "en":
        return text  # no translation needed

    try:
        tokenizer, model = load_marian_model(lang)
        inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
        outputs = model.generate(**inputs)
        translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated
    except Exception:
        return text  # fallback if model not available or translation fails
    
def find_pii_patterns(text: Any, lang: str) -> Dict[str, List[str]]:
    """
    Find PII patterns in text using regex and Named Entity Recognition (NER).
    Automatically translates non-English text to English before analysis.

    Regex:
      - Emails
      - Phone numbers
      - IP addresses
      - Credit card numbers
      - Street-style addresses

    NER:
      - PERSON (names)

    Args:
        text (Any): Input text to scan.

    Returns:
        Dict[str, List[str]]: Dictionary with PII type as key and matches as values.
    """
    if not isinstance(text, str):
        return {}

    patterns = {
        "email": r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
        "phone": (
            r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{3}\)?|\d{3})"
            r"[-.\s]?\d{3}[-.\s]?\d{4}\b"
        ),
        "ip": r"\b(?:\d{1,3}\.){3}\d{1,3}\b",
        "credit_card": r"\b(?:\d[ -]*?){13,16}\b",
        "address": r"\b\d{1,5}\s+\w+(?:\s\w+)?\s+(St|Street|Ave|Avenue|Rd|Road|Blvd|Boulevard|Dr|Drive|Ln|Lane)\b",
    }

    found = {}
    for key, pattern in patterns.items():
        matches = re.findall(pattern, text)
        if matches:
            found[key] = matches
    
    nlp = nlp_en if lang == "en" else nlp_de if lang == "de" else None
    if not nlp:
        return found

    # NER-based detection
    doc = nlp(text)
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    if names:
        found["names"] = names

    return found

def redact_pii(text: Any, lang: str) -> str:
    """
    Redact PII from text using regex and Named Entity Recognition (NER).

    Regex:
      - Emails
      - Phone numbers
      - IP addresses
      - Credit card numbers
      - Street-style addresses

    NER:
      - PERSON (names)

    Args:
        text (Any): Input text to redact.
        lang (str): Language code ('en' or 'de') for appropriate NER model.

    Returns:
        str: Text with PII replaced by placeholders.
    """
    if not isinstance(text, str):
        return ""

    redacted = text  # <- Define redacted here

    patterns = {
        "email": r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
        "phone": (
            r"\b(\+?\d{1,3}[-.\s]?)?(\(?\d{3}\)?|\d{3})"
            r"[-.\s]?\d{3}[-.\s]?\d{4}\b"
        ),
        "ip": r"\b(?:\d{1,3}\.){3}\d{1,3}\b",
        "credit_card": r"\b(?:\d[ -]*?){13,16}\b",
        "address": r"\b\d{1,5}\s+\w+(?:\s\w+)?\s+(St|Street|Ave|Avenue|Rd|Road|Blvd|Boulevard|Dr|Drive|Ln|Lane)\b",
    }

    for key, pattern in patterns.items():
        redacted = re.sub(pattern, f"[{key.upper()}_REDACTED]", redacted)

    nlp = nlp_en if lang == "en" else nlp_de if lang == "de" else None
    if not nlp:
        return redacted

    # NER-based redaction
    doc = nlp(redacted)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            redacted = redacted.replace(ent.text, "<NAME>")

    return redacted

In [3]:
#load the dataset
raw_multi_df = pd.read_csv("../data/raw/aa_dataset-tickets-multi-lang-5-2-50-version.csv")
raw_multi_df.sample(n=1)

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
6319,Website Outage Encountered Today,"The agency's website has been experiencing repeated outages, potentially due to recent server updates. Restarting the servers and clearing the cache did not resolve the issue. Our team is actively investigating the root cause and working towards a solution. We apologize for any inconvenience caused and appreciate your patience during this time. Please feel free to reach out with any questions or concerns.",Thank you for notifying us. We apologize for the inconvenience. Please contact the service desk agent for further assistance regarding the website outage.,Incident,IT Support,medium,en,52,Outage,Disruption,Recovery,Maintenance,IT,Tech Support,NaN,NaN


In [4]:
#drop the non-english records
en_de_only_df = raw_multi_df[(raw_multi_df['language'] == "en") 
                             | (raw_multi_df['language'] == 'de')].copy()

In [5]:
#add combined text column
df_combo_text = combine_text_columns(en_de_only_df, ["subject", "body"])
df_combo_text.sample(n=1)

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,Combined_Text_Col
24412,Seek Assistance for Integration into SaaS Platform,"Hello, I am writing to request support for integrating our SaaS platform into our affected products. The integration aims to enhance project management functionality, ensuring seamless compatibility and improved workflow. I would greatly appreciate any guidance on the necessary steps to achieve this integration. Please let me know if there are any additional requirements or information you need from me. I look forward to hearing from you soon.","Hello [Name], thank you for reaching out to our service desk regarding integration support for our SaaS platform with your products. We appreciate your interest in enhancing project management functionality and improving workflow compatibility. To proceed with the integration, we will need specific information about the products you wish to integrate with. Could you please provide us with details about your current system, including any relevant API documentation and technical specifications? Additionally, it would be helpful if you could let us know about any specific features or functionalities you require. We look forward to assisting you with this process.",Change,Technical Support,medium,en,400,Feature,Documentation,Tech Support,Feedback,NaN,NaN,NaN,NaN,"Seek Assistance for Integration into SaaS Platform Hello, I am writing to request support for integrating our SaaS platform into our affected products. The integration aims to enhance project management functionality, ensuring seamless compatibility and improved workflow. I would greatly appreciate any guidance on the necessary steps to achieve this integration. Please let me know if there are any additional requirements or information you need from me. I look forward to hearing from you soon."


In [6]:
#now check for PII ->
df_combo_text['PII_found'] = df_combo_text.apply(
    lambda row: find_pii_patterns(row["Combined_Text_Col"],
                                   row["language"]), axis=1)

In [7]:
#now redact PII
df_combo_text['Redacted_Text'] = df_combo_text.apply(
    lambda row: redact_pii(row["Combined_Text_Col"],
                                   row["language"]), axis=1)

In [14]:
#running this cell a few times to assess
df_combo_text[["Combined_Text_Col", "PII_found", "Redacted_Text"]].sample(n=1)

,Combined_Text_Col,PII_found,Redacted_Text
20798,"Datenschutzlücken im Krankenhaus-IT-System Es wurden Datenschutzlücken in den IT-Systemen des Krankenhauses festgestellt, die sensible medizinische Daten gefährden könnten. Aufgrund von Software-Integrationsschwerpunkten und veralteten Sicherheitsprotokollen sind diese Probleme entstanden. Bislang wurden Feuerwal-Einstellungen aktualisiert, Softwareaudits durchgeführt und Zugriffsprotokolle überprüft, um die Ursache zu identifizieren. Bitte engagieren Sie sich, um die Angelegenheit baldigst zu lösen und die Integrität unserer medizinischen Daten zu gewährleisten.",{},"Datenschutzlücken im Krankenhaus-IT-System Es wurden Datenschutzlücken in den IT-Systemen des Krankenhauses festgestellt, die sensible medizinische Daten gefährden könnten. Aufgrund von Software-Integrationsschwerpunkten und veralteten Sicherheitsprotokollen sind diese Probleme entstanden. Bislang wurden Feuerwal-Einstellungen aktualisiert, Softwareaudits durchgeführt und Zugriffsprotokolle überprüft, um die Ursache zu identifizieren. Bitte engagieren Sie sich, um die Angelegenheit baldigst zu lösen und die Integrität unserer medizinischen Daten zu gewährleisten."


In [9]:
df_combo_text["language"].value_counts()

language
en    16338
de    12249
Name: count, dtype: int64

In [ ]:
#now translate combined text field to english -> I let this run for about 4hrs and it still has not completed.
#df_combo_text["translated_text"] = df_combo_text["Combined_Text_Col"].apply(translate_to_english)

### Lessons Learned:
1. In viewing the "Answer" column I can see that the data was previously redacted, however it doesn't seem that "body" and "subject" were redacted.

2. The redaction function will get added to the pipeline to help reduce pii in new data (production)

3. I experimented the spacy Small models but had too many misclassifications, the Large models seem to perform better.

4. One thing that increases missclassifications is the lack of context - for example the "subject" column alone may be too short for NER to be effective. I decided to combine the "subject" and "body" columns.

5. Sometimes product is misclassified by Spacy NER - it's recognized as person entity

6. One enhancement I would make is translation - the traslation function takes hours to implement since we have 12k german records.